In [1]:
import os
import zipfile

with zipfile.ZipFile('/content/drive/MyDrive/Datasets/SOURCE4/cvl-database-1-1.zip' ,'r') as cvl_f:
  cvl_f.extractall('/content/')

In [7]:
WORDS_TRAIN_DIR = '/content/cvl-database-1-1/trainset/words'
WORDS_TRAIN_SUBDIRS = [d for d in os.listdir(WORDS_TRAIN_DIR) if os.path.isdir(os.path.join(WORDS_TRAIN_DIR, d))]

WORDS_SUBDIRS_FILES = {}
for d in WORDS_TRAIN_SUBDIRS:
  WORDS_SUBDIRS_FILES[d] = [f for f in os.listdir(os.path.join(WORDS_TRAIN_DIR, d)) if f[-4:] == ".tif"]

In [8]:
print(WORDS_SUBDIRS_FILES)

{'0015': ['0015-1-5-7-country.tif', '0015-2-2-5-deserves.tif', '0015-1-3-11-much.tif', '0015-7-2-7-because.tif', '0015-2-5-6-out.tif', '0015-3-4-3-not.tif', '0015-6-0-0-Werd.tif', '0015-7-2-10-had.tif', "0015-2-1-3-rebel's.tif", '0015-8-2-4-my.tif', '0015-6-1-1-doch.tif', '0015-4-4-2-in.tif', '0015-3-7-0-Volltransistor-Rechenautomat.tif', '0015-7-4-8-art.tif', '0015-1-7-1-has.tif', '0015-8-2-7-brain.tif', '0015-7-0-5-love.tif', '0015-6-7-0-Es.tif', '0015-8-5-8-silently.tif', '0015-3-6-3-Wiener.tif', '0015-3-2-3-at.tif', '0015-1-1-9-fixed.tif', '0015-3-1-2-on.tif', '0015-7-5-8-and.tif', '0015-1-7-3-opened.tif', '0015-1-1-6-instead.tif', '0015-3-2-9-Technology.tif', '0015-4-1-6-and.tif', '0015-3-5-10-be.tif', '0015-6-6-5-der.tif', '0015-8-5-0-dank.tif', '0015-1-5-4-notion.tif', '0015-2-2-6-that.tif', '0015-4-4-3-a.tif', '0015-4-3-5-than.tif', '0015-4-1-10-the.tif', '0015-8-5-9-over.tif', '0015-6-3-6-gehn.tif', '0015-4-4-4-state.tif', '0015-3-6-2-a.tif', '0015-2-1-2-a.tif', '0015-7-0-11-i

In [9]:
import re

def extract_w(filename):
    pattern = r'\d+-\d+-\d+-\d+-(.+)\.tif'

    match = re.search(pattern, filename)

    if match:
        words = match.group(1)
        return words
    else:
        return None


In [10]:
WORDS_FROM_SUBDIRS = {}
for d in [*WORDS_SUBDIRS_FILES]:
  WORDS_FROM_SUBDIRS[d] = [extract_w(f) for f in WORDS_SUBDIRS_FILES[d]]

In [11]:
import pandas as pd

freq_df = pd.read_csv('/content/drive/MyDrive/Datasets/etc/unigram_freq.csv')
freq_df = freq_df.sort_values(by=['count'], axis=0, ascending=False)
freq_words = freq_df['word'].to_numpy()
freq_df.head()

,word,count
0,the,23135851162
1,of,13151942776
2,and,12997637966
3,to,12136980858
4,a,9081174698


In [12]:
!pip install english-words lxml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 34.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for english-words: filename=english_words-2.0.1-py3-none-any.whl size=8196236 sha256=7492f61e13e7590403188a4edad32bc3b547bef276a43f8a1d2d17580df11fb9
  Stored in directory: /root/.cache/pip/wheels/f0/e6/d9/16a123647999fe535f03a36e7af23eef203736d84c7ca25b0b
Successfully built english-words


In [13]:
from english_words import get_english_words_set
import numpy as np

words_list = list(get_english_words_set(['web2']))

SOURCE2_WORDS_PATH = '/content/drive/MyDrive/Datasets/SOURCE2/words_new.txt'
words_txt = []
with open(SOURCE2_WORDS_PATH, 'r') as w_f:
    lines = w_f.readlines()
    for line in lines:
        if line[0] == "#":
            continue
        spl = line.strip().split(" ")
        words_txt.append(spl[-1])

unique_source2_words = np.unique(np.array(words_txt))
freq_without_source2 = [w for w in freq_words if w not in unique_source2_words]
freq_source2_intersection = [w for w in freq_words if w in unique_source2_words]


In [14]:
model_vocab_size = 50250

freq_new_size = model_vocab_size - len(unique_source2_words)
cropped_freq = freq_without_source2[:freq_new_size]

unique_source2_words, source2_counts = np.unique(words_txt, return_counts=True)
sorted_pairs = sorted(zip(unique_source2_words, source2_counts), key=lambda x: x[1], reverse=True)
sorted_unique, sorted_counts = zip(*sorted_pairs)

sorted_unique = list(sorted_unique)
sorted_counts = list(sorted_counts)

total_counts = sum(sorted_counts)
probabilities = [count / total_counts for count in sorted_counts]

In [15]:
WORDS_IN_DICT = {}
WORDS_NOT_IN_DICT = {}

for d in [*WORDS_FROM_SUBDIRS]:
  WORDS_IN_DICT[d] = [w for w in WORDS_FROM_SUBDIRS[d] if w in cropped_freq or w in sorted_unique]
  WORDS_NOT_IN_DICT[d] = [w for w in WORDS_FROM_SUBDIRS[d] if w not in cropped_freq and w not in sorted_unique]

In [16]:
overall_valid_train = sum([len(WORDS_IN_DICT[d]) for d in [*WORDS_IN_DICT]])

In [17]:
import random
import time
import shutil
from PIL import Image

out_train_dir = '/content/SOURCE3/train'
out_test_dir = '/content/SOURCE3/test'

train_csv = '/content/SOURCE3/train.csv'
test_csv = '/content/SOURCE3/test.csv'

os.makedirs(out_train_dir, exist_ok=True)
os.makedirs(out_test_dir, exist_ok=True)

def generate_filename(frmt='jpg'):
    return str(time.time()) + '_' + str(random.randint(100, 999)) + '.' + frmt

In [18]:
WORDS_TEST_DIR = '/content/cvl-database-1-1/testset/words'
WORDS_TESR_SUBDIRS = [d for d in os.listdir(WORDS_TEST_DIR) if os.path.isdir(os.path.join(WORDS_TEST_DIR, d))]

TEST_SUBDIRS_FILES = {}
for d in WORDS_TESR_SUBDIRS:
  TEST_SUBDIRS_FILES[d] = [f for f in os.listdir(os.path.join(WORDS_TEST_DIR, d)) if f[-4:] == ".tif"]

In [19]:
WORDS_FROM_TEST_SUBDIRS = {}
for d in [*TEST_SUBDIRS_FILES]:
  WORDS_FROM_TEST_SUBDIRS[d] = [extract_w(f) for f in TEST_SUBDIRS_FILES[d]]

In [20]:
TEST_WORDS_IN_DICT = {}

for d in [*WORDS_FROM_TEST_SUBDIRS]:
  TEST_WORDS_IN_DICT[d] = [w for w in WORDS_FROM_TEST_SUBDIRS[d] if w in cropped_freq or w in sorted_unique]


In [21]:
overall_valid_test = sum([len(TEST_WORDS_IN_DICT[d]) for d in [*TEST_WORDS_IN_DICT]])
overall_valid = overall_valid_test + overall_valid_test

tagret_test_len = int(overall_valid * 0.2)
min_test_len = tagret_test_len - 260
max_test_len = tagret_test_len + 260

test_dirnames = []
train_dirnames = []

cur_test_len = 0
achieved_target = False

for d in [*TEST_WORDS_IN_DICT]:
  if not achieved_target:
    cur_test_len += len(TEST_WORDS_IN_DICT[d])
    test_dirnames.append(d)

    if cur_test_len > min_test_len and cur_test_len < max_test_len:
      achieved_target = True
    elif cur_test_len > min_test_len and cur_test_len > max_test_len:
      raise ValueError("Unexpected comparison situation")

  else:
    train_dirnames.append(d)



In [22]:
print(cur_test_len/overall_valid)

0.19870948716508455


In [23]:
filenames_list = []
transcriptions_list = []
files_writers_ids = []
initial_filenames_list = []

def process_images(words_dir, dir_names, words_from_subdirs, subdirs_files, out_dir, cropped_freq, sorted_unique):
    for d in dir_names:
        for idx in range(len(words_from_subdirs[d])):
            word = words_from_subdirs[d][idx]
            if word in cropped_freq or word in sorted_unique:
                src_path = os.path.join(words_dir, d, subdirs_files[d][idx])

                with Image.open(src_path) as img:
                    out_filename = generate_filename()
                    out_path = os.path.join(out_dir, out_filename)

                    # Ensure unique filenames
                    while os.path.exists(out_path):
                        out_path = os.path.join(out_dir, generate_filename())

                    img.convert('RGB').save(out_path, 'JPEG')

                    filenames_list.append(out_filename)
                    transcriptions_list.append(word)
                    files_writers_ids.append(d)
                    initial_filenames_list.append(subdirs_files[d][idx])


def save_to_csv_and_archive(filenames_list, transcriptions_list, files_writers_ids,
                            initial_filenames_list, csv_path, archive_path, out_dir):
    df = pd.DataFrame({
        'filename': filenames_list,
        'transcription': transcriptions_list,
        'writer_id': files_writers_ids,
        'initial_filename': initial_filenames_list
    })

    df.to_csv(csv_path, index=False)

    shutil.make_archive(archive_path, "zip", out_dir)

In [24]:
process_images(
    WORDS_TEST_DIR, test_dirnames, WORDS_FROM_TEST_SUBDIRS, TEST_SUBDIRS_FILES, out_test_dir, cropped_freq, sorted_unique
)

save_to_csv_and_archive(filenames_list, transcriptions_list, files_writers_ids,
                        initial_filenames_list, test_csv, "/content/SOURCE3/test", out_test_dir)

filenames_list = []
transcriptions_list = []
files_writers_ids = []
initial_filenames_list = []


process_images(
    WORDS_TRAIN_DIR, [*WORDS_FROM_SUBDIRS], WORDS_FROM_SUBDIRS, WORDS_SUBDIRS_FILES, out_train_dir, cropped_freq, sorted_unique
)

process_images(
    WORDS_TEST_DIR, train_dirnames, WORDS_FROM_TEST_SUBDIRS, TEST_SUBDIRS_FILES, out_train_dir, cropped_freq, sorted_unique
)

save_to_csv_and_archive(filenames_list, transcriptions_list, files_writers_ids,
                        initial_filenames_list, train_csv, "/content/SOURCE3/train", out_train_dir)



In [162]:
from lxml import etree

def extract_bbox_coords(points):
    xs = []
    ys = []

    if not points:  # If points list is empty, return None
        print("No points found for bounding box.")
        return None

    for point in points:
        x = point.get('x')
        y = point.get('y')

        if x is None or y is None:  # Check if 'x' or 'y' attributes are missing
            print(f"Point missing x or y attribute: {point.attrib}")
            return None

        xs.append(float(x))
        ys.append(float(y))

    return [min(xs), min(ys), max(xs), max(ys)]

def extract_annotations(xml_filepath):
    ns = {'pc': 'http://schema.primaresearch.org/PAGE/gts/pagecontent/2010-03-19'}

    with open(xml_filepath, 'rb') as file:
        tree = etree.parse(file)

    handwriting_crop_bbox = []
    page_id = None

    for attr_region in tree.findall('.//pc:AttrRegion[@attrType="3"][@fontType="2"]', namespaces=ns):
        page_id = attr_region.get('id')
        if page_id is None:
            print("No page ID found.")
            page_id = None

        min_area_rect = attr_region.find('.//pc:minAreaRect', namespaces=ns)
        if min_area_rect is not None:
            handwriting_crop_bbox = extract_bbox_coords(min_area_rect.findall('.//pc:Point', namespaces=ns))
        else:
            print(f"No 'minAreaRect' found for page ID: {page_id}")
            handwriting_crop_bbox = None

    annotated_regions = []
    annotated_regions_contents = []

    for region in tree.findall('.//pc:AttrRegion[@attrType="2"][@fontType="2"]', namespaces=ns):
        region_id = region.get('id')
        if region_id is None:
            print("No region ID found.")
            region_id = None

        median_word_height = region.get('medianWordHeight')
        if median_word_height is None:
            print(f"No medianWordHeight found for region ID: {region_id}")
            median_word_height = None
        else:
            median_word_height = float(median_word_height)

        font_angle_rad = region.get('fontAngleRad')
        if font_angle_rad is None:
            print(f"No fontAngleRad found for region ID: {region_id}")
            font_angle_rad = None
        else:
            font_angle_rad = float(font_angle_rad)

        min_area_rect = region.find('.//pc:minAreaRect', namespaces=ns)
        if min_area_rect is not None:
            region_bbox = extract_bbox_coords(min_area_rect.findall('.//pc:Point', namespaces=ns))
        else:
            print(f"No 'minAreaRect' found for region ID: {region_id}")
            region_bbox = None

        annotated_regions.append({
            "region_id": region_id,
            "median_word_height": median_word_height,
            "font_angle_rad": font_angle_rad,
            "region_bbox": region_bbox,
        })

        region_contents = {
            "transcriptions": [],
            "ids": [],
            "bboxes": []
        }

        # Find all subregions within the region that contain transcriptions
        for subregion in region.findall('.//pc:AttrRegion[@text]', namespaces=ns):
            text = subregion.get('text')
            if text is None:
                print(f"No text found in subregion for region ID: {region_id}")
                text = None

            id = subregion.get('id')
            if id is None:
                print(f"No ID found for subregion in region ID: {region_id}")
                id = None

            min_area_rect = subregion.find('.//pc:minAreaRect', namespaces=ns)
            if min_area_rect is not None:
                bbox = extract_bbox_coords(min_area_rect.findall('.//pc:Point', namespaces=ns))
            else:
                print(f"No 'minAreaRect' found for subregion ID: {id}")
                bbox = None  # Set to None if not found

            region_contents["transcriptions"].append(text)
            region_contents["ids"].append(id)
            region_contents["bboxes"].append(bbox)

        annotated_regions_contents.append(region_contents)

    return {
        "page_id": page_id,
        "handwriting_crop_bbox": handwriting_crop_bbox,
        "regions": annotated_regions,
        "regions_contents": annotated_regions_contents,
    }


In [140]:
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt

def draw_bboxes(image_path, annotations, output_path="/content/annotated_image.tif"):
    image = Image.open(image_path)
    draw = ImageDraw.Draw(image)

    if annotations["handwriting_crop_bbox"]:
        bbox = annotations["handwriting_crop_bbox"]
        draw.rectangle(bbox, outline="red", width=2)

    for region in annotations["regions"]:
        if region["region_bbox"]:
            bbox = region["region_bbox"]
            draw.rectangle(bbox, outline="blue", width=2)

    for region_contents in annotations["regions_contents"]:
        for bbox in region_contents["bboxes"]:
            if bbox:
                draw.rectangle(bbox, outline="green", width=2)

    # Save the image
    image.save(output_path)
    print(f"Annotated image saved at {output_path}")

In [163]:
def label_encoding(file_path):
    with open(file_path, 'rb') as file:
        content = file.read()

    content = content.replace(b'encoding="UTF-16"', b'encoding="UTF-8"')

    with open(file_path, 'wb') as file:
        file.write(content)



In [164]:
def crop_from_bbox(image_path, bbox, out_path):
  image = Image.open(image_path).crop(bbox)
  image.save(out_path, "JPEG", quality=95)

In [169]:
train_pages_annotations = {}
test_pages_annotations = {}

annotations_dir = '/content/SOURCE3/detection'
annotations_train_dir = '/content/SOURCE3/detection/train'
annotations_test_dir = '/content/SOURCE3/detection/test'

os.makedirs(annotations_dir, exist_ok=True)
os.makedirs(annotations_train_dir, exist_ok=True)
os.makedirs(annotations_test_dir, exist_ok=True)

train_xml_dir = '/content/cvl-database-1-1/trainset/xml'
test_xml_dir = '/content/cvl-database-1-1/testset/xml'

train_pages_dir = '/content/cvl-database-1-1/trainset/pages'
test_pages_dir = '/content/cvl-database-1-1/testset/pages'

# Throwing out german texts (most of them are corrupted, not relevant)
train_xml_files = [f for f in os.listdir(train_xml_dir)\
                   if f[-4:] == '.xml' and '-6_attributes.xml'\
                   not in f and '-3_attributes.xml' not in f]
# There are two blank pages left by writer, one corrupted
merged_xml_files = [f for f in os.listdir(test_xml_dir)\
                   if f[-4:] == '.xml' and '-6_attributes.xml'\
                   not in f and '-3_attributes.xml' not in f \
                    and '0431-3' not in f and '0431-4' not in f \
                    and '0161-4' not in f and '0161-2' not in f\
                    and '0161-3' not in f and '0161-6' not in f]

def process_page_and_xml(xml_files_list, xml_dir, pages_dir, annotations_dir, annotations_dict):
  for xml_f in xml_files_list:
    xml_filename = os.path.join(xml_dir, xml_f)
    print(xml_filename)
    label_encoding(xml_filename)
    annotation = extract_annotations(xml_filename)

    crop_bbox = annotation['handwriting_crop_bbox']
    page_id = annotation['page_id']

    page_filepath = os.path.join(pages_dir, page_id + '.tif')
    out_page_filepath = os.path.join(annotations_dir, page_id + '.jpg')

    crop_from_bbox(page_filepath, crop_bbox, out_page_filepath)

    annotations_dict[page_id] = {"regions": annotation["regions"],
                                 "regions_contents": annotation["regions_contents"]}





In [134]:
draw_bboxes('/content/cvl-database-1-1/trainset/pages/0001-1.tif', annotation_info)

Annotated image saved at /content/annotated_image.tif


In [135]:
merged_xml_writers = set([f[:4] for f in merged_xml_files])
merged_dirnames = set(test_dirnames + train_dirnames)
print(len(merged_xml_writers), len(merged_dirnames), len(merged_xml_writers.intersection(merged_dirnames)))

train_xml_writers = set([f[:4] for f in train_xml_files])
print(len([*WORDS_FROM_SUBDIRS]), len(train_xml_writers), len(train_xml_writers.intersection(set([*WORDS_FROM_SUBDIRS]))))

283 283 283
27 27 27


In [136]:
print(len(test_dirnames))

112


In [170]:
test_files_from_merged = []
train_files_from_merged = []

for xml_f in merged_xml_files:
  xml_writer_id = xml_f[:4]

  if xml_writer_id in test_dirnames:
    test_files_from_merged.append(xml_f)
  elif xml_writer_id in train_dirnames:
    train_files_from_merged.append(xml_f)
  else:
    raise ValueError("XML file doesn't belong to any valid writer id's")


In [172]:
print(len(test_files_from_merged), len(train_files_from_merged))

336 510


In [ ]:
import json

process_page_and_xml(
    train_xml_files, train_xml_dir, train_pages_dir,
    annotations_train_dir, train_pages_annotations)

process_page_and_xml(
    train_files_from_merged, test_xml_dir, test_pages_dir,
    annotations_train_dir, train_pages_annotations)




train_annotations_json = '/content/SOURCE3/detection/train_annotations.json'
test_annotations_json = '/content/SOURCE3/detection/test_annotations.json'


with open(train_annotations_json, 'w') as f:
  json.dump(train_pages_annotations, f)

process_page_and_xml(
    test_files_from_merged, test_xml_dir, test_pages_dir,
    annotations_test_dir, test_pages_annotations)


with open(test_annotations_json, 'w') as f:
  json.dump(test_pages_annotations, f)


In [175]:
print(f'Percentage of valid from init. testset: {len(merged_xml_files)/len(os.listdir(test_xml_dir)):.2f}')
print(f'Percentage of valid from init. train: {len(train_xml_files)/len(os.listdir(train_xml_dir)):.2f}')

Percentage of valid from init. testset: 0.60
Percentage of valid from init. train: 0.71


In [177]:
shutil.make_archive('/content/SOURCE3/detection', "zip", '/content/SOURCE3/detection/')

'/content/SOURCE3/detection.zip'